# Inspection of Files


In [1]:
import sys
import fsspec
import xarray as xr
import hvplot.xarray
import zarr

ModuleNotFoundError: No module named 'xarray'

In [7]:
print("Python : ", sys.version)
print("fsspec : ", fsspec.__version__)
print("zarr   : ", zarr.__version__)

Python :  3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC 11.3.0]
fsspec :  2023.3.0+9.g5920300
zarr   :  2.13.3


In [2]:
fs = fsspec.filesystem('s3', anon=True)

In [3]:
flist = fs.ls('s3://noaa-nwm-retrospective-2-1-pds/')
flist

['noaa-nwm-retrospective-2-1-pds/forcing',
 'noaa-nwm-retrospective-2-1-pds/index.html',
 'noaa-nwm-retrospective-2-1-pds/model_output']

In [4]:
flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/*')
print(flist[0])
print(flist[-1])

noaa-nwm-retrospective-2-1-pds/model_output/1979
noaa-nwm-retrospective-2-1-pds/model_output/2020


In [5]:
flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/1979/*LDAS*')
flist[0]

'noaa-nwm-retrospective-2-1-pds/model_output/1979/197902010300.LDASOUT_DOMAIN1.comp'

In [6]:
flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/2020/*LDAS*')
flist[-1]

'noaa-nwm-retrospective-2-1-pds/model_output/2020/202012312100.LDASOUT_DOMAIN1.comp'

Okay, so at this point we've learned that we have 3-hourly output over roughly 40 years

In [7]:
# %%time
# flist = fs.glob('noaa-nwm-retrospective-2-1-pds/model_output/*/*LDAS*')   # this is slow
40 * 365 * 24 / 3

116800.0

In [8]:
flist[0]

'noaa-nwm-retrospective-2-1-pds/model_output/2020/202001010000.LDASOUT_DOMAIN1.comp'

So about 117,000 NetCDF files! 

Let's check one out.  Although it's not super efficient, we can open a NetCDF file on S3 as a virtual file object with `fs.open(s3_url_of_netcdf_file)`.  If we open a dataset in xarray using `chunks=` we are telling xarray to use Dask, and `chunks={}` means use the native chunking in the NetCDF file

In [9]:
ds = xr.open_dataset(fs.open(flist[0]), chunks={})

In [10]:
ds

<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, x: 4608, y: 3840,
                     soil_layers_stag: 4, vis_nir: 2)
Coordinates:
  * time            (time) datetime64[ns] 2020-01-01
  * reference_time  (reference_time) datetime64[ns] 2019-10-01
  * x               (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y               (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Dimensions without coordinates: soil_layers_stag, vis_nir
Data variables: (12/21)
    crs             |S1 ...
    COSZ            (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    FSA             (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    FIRA            (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    HFX             (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    LH              (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ...              ...
    FSNO            (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ACSNOM          (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ACCET           (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ALBSND          (time, y, vis_nir, x) float64 dask.array<chunksize=(1, 3840, 2, 4608), meta=np.ndarray>
    ALBSNI          (time, y, vis_nir, x) float64 dask.array<chunksize=(1, 3840, 2, 4608), meta=np.ndarray>
    QRAIN           (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
Attributes:
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    model_initialization_time:  2019-10-01_00:00:00
    model_output_valid_time:    2020-01-01_00:00:00
    model_total_valid_times:    736
    Conventions:                CF-1.6
    code_version:               v5.2.0-beta2
    model_output_type:          land
    model_configuration:        retrospective
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    GDAL_DataType:              Generic

In [11]:
ds.data_vars

Data variables:
    crs       |S1 ...
    COSZ      (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    FSA       (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    FIRA      (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    HFX       (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    LH        (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    EDIR      (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    ALBEDO    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    UGDRNOFF  (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    TRAD      (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    SOIL_W    (time, y, soil_layers_stag, x) float64 dask.array<chunksize=(1, 3840, 4, 4608), meta=np.ndarray>
    SOIL_M    (time, y, soil_layers_stag,

In [12]:
ds = ds[['ACCET', 'SNEQV', 'FSNO']]

In [13]:
ds.data_vars

Data variables:
    ACCET    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    SNEQV    (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>
    FSNO     (time, y, x) float64 dask.array<chunksize=(1, 3840, 4608), meta=np.ndarray>

In [14]:
ds['ACCET']

<xarray.DataArray 'ACCET' (time: 1, y: 3840, x: 4608)>
dask.array<open_dataset-578a09b313064862347dabd291e2986dACCET, shape=(1, 3840, 4608), dtype=float64, chunksize=(1, 3840, 4608), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    long_name:       Accumulated total ET
    units:           mm
    grid_mapping:    crs
    valid_range:     [  -100000 100000000]
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...

In [15]:
ds.ACCET

<xarray.DataArray 'ACCET' (time: 1, y: 3840, x: 4608)>
dask.array<open_dataset-578a09b313064862347dabd291e2986dACCET, shape=(1, 3840, 4608), dtype=float64, chunksize=(1, 3840, 4608), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    long_name:       Accumulated total ET
    units:           mm
    grid_mapping:    crs
    valid_range:     [  -100000 100000000]
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...

The data is chunked as full spatial domain and 1 time step, with about 135MB chunk size.   This is actually great for visualization of maps at specific time steps or for calculations that involve the entire dataset. So kerchunking this data would be a nice first step. 

In [16]:
%%time
da = ds.ACCET.load()

CPU times: user 765 ms, sys: 190 ms, total: 955 ms
Wall time: 3.24 s


In [17]:
da

<xarray.DataArray 'ACCET' (time: 1, y: 3840, x: 4608)>
array([[[         nan,          nan,          nan, ..., 194.61999565,
         197.71999558, 198.85999556],
        [         nan,          nan,          nan, ..., 191.09999573,
         193.14999568, 197.60999558],
        [         nan,          nan,          nan, ..., 186.99999582,
         197.26999559, 207.02999537],
        ...,
        [         nan,          nan,          nan, ...,   3.62999992,
           3.70999992,   3.74999992],
        [         nan,          nan,          nan, ...,   3.67999992,
           3.68999992,   3.72999992],
        [         nan,          nan,          nan, ...,   3.66999992,
           3.68999992,   3.69999992]]])
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    long_name:       Accumulated total ET
    units:           mm
    grid_mapping:    crs
    valid_range:     [  -100000 100000000]
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...

In [18]:
da.hvplot(x='x', y='y', rasterize=True, cmap='turbo', data_aspect=1)

:DynamicMap   [time]
   :Image   [x,y]   (ACCET)